In [1]:
import numpy as np
import cv2
from copy import deepcopy
from PIL import Image
from keras.models import load_model
from string import ascii_uppercase

def ocr(input_im):
    input_im = cv2.cvtColor(input_im,cv2.COLOR_GRAY2RGB)
    input_im=cv2.resize(input_im,(150,150),interpolation = cv2.INTER_AREA)
    input_im = input_im.reshape(1,150,150,3)
    input_im = input_im.astype('float32')
    input_im/=255
    classifier = load_model('C:/Users/JOY/DeepLearningCV/10. Data Augmentation/ocr_augmented_91%.h5')
    res = classifier.predict_classes(input_im, 1, verbose = 0)[0]
    dict=[]
    for i in range(0,10):
        dict.append(str(i))
    for c in ascii_uppercase:
        dict.append(str(c))
    return dict[res]

def x_cord_contour(contours):
    #Returns the X cordinate for the contour centroid
    if cv2.contourArea(contours) > 1:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        return 100000
def preprocess(img):
    """This function takes an image, applies blurring, uses sobel
    to get horizontal lines. It then returns the binarized image"""
    cv2.imshow("Input",img)
    #cv2.waitKey(0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 30, 200)
   
    
    cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU,gray)
    #cv2.imshow("Threshold",gray)
    #cv2.waitKey(0)
    return gray

def cleanPlate(plate):
    """This function gets the countours that most likely resemeber the shape
    of a license plate"""    
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    _, thresh=cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if contours:
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)

        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        x,y,w,h = cv2.boundingRect(max_cnt)

        if not ratioCheck(max_cntArea,w,h):
            return plate,None

        cleaned_final = thresh[y:y+h, x:x+w]
        #cv2.imshow("Function Test", cleaned_final)
        return cleaned_final,[x,y,w,h]

    else:
        return plate, None

def extract_contours(threshold_img):
    """This function returns the extracted contours"""
    element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
    morph_img_threshold = threshold_img.copy()
    cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
    #cv2.imshow("Morphed", morph_img_threshold)
    #cv2.waitKey(0)

    contours, hierarchy= cv2.findContours(morph_img_threshold,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)
    print("Number of Contours found = " + str(len(contours)))
    cv2.drawContours(morph_img_threshold, contours, -1, (0, 255, 0), 3)
    return contours


def ratioCheck(area, width, height):
    """This function inspects the ratio of the contour to ensure it meets the requirements
    suitable to a real license plate"""
    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio

    aspect = 4.7272
    min = 15 * aspect * 15  # minimum area
    max = 125 *aspect * 125  # maximum area

    rmin = 3
    rmax = 6

    if (area < min or area > max) or (ratio < rmin or ratio > rmax):
        return False
    return True

def isMaxWhite(plate):
    """Checks the average color of the potential plate and if there is more
    white than black colors it returns true"""
    avg = np.mean(plate)
    if(avg>=115):
        return True
    else:
        return False

def validateRotationAndRatio(rect):
    """Checks the angle of the rectangle potential license plate"""
    (x, y), (width, height), rect_angle = rect

    if(width>height):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle

    if angle>15:
        return False

    if height == 0 or width == 0:
        return False

    area = height*width
    if not ratioCheck(area,width,height):
        return False
    else:
        return True

def cleanAndRead(img,contours):
    """Takes the extracted contours and once it passes the rotation
    and ratio checks it passes the potential license plate to PyTesseract for OCR reading"""
    for i,cnt in enumerate(contours):
        min_rect = cv2.minAreaRect(cnt)

        if validateRotationAndRatio(min_rect):

            x,y,w,h = cv2.boundingRect(cnt)
            plate_img = img[y:y+h,x:x+w]

            if(isMaxWhite(plate_img)):
                clean_plate, rect = cleanPlate(plate_img)
                if rect:
                    x1,y1,w1,h1 = rect
                    x,y,w,h = x+x1,y+y1,w1,h1
                    #cv2.imshow("Cleaned Plate", clean_plate)
                    #cv2.waitKey(0)
                    plate_im=clean_plate
                    #plate_im = Image.fromarray(clean_plate)
                   
                    plate_im=cv2.resize(plate_im, (400, 200), interpolation = cv2.INTER_CUBIC)
                    #cv2.imshow('clean_plate_resize',plate_im)
                    #cv2.waitKey(0)
                    contours_list, hierarchy = cv2.findContours(plate_im, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                    contours_sorted = sorted(contours_list, key = x_cord_contour, reverse = False)
                    print("Number of Contours found in Clean Plate = " + str(len(contours)))
                    for c in contours_sorted:
                        x,y,w,h = cv2.boundingRect(c)
                        #print(w,h)
                        if h<100 or w<10:
                            continue
                        if h==200 and w==400:
                            continue
                        cv2.drawContours(plate_im, [c], -1, (0,255,0), 3)
                        char_img=plate_im[y:y+h , x:x+w]
                        #cv2.imshow('char',char_img)                        
                        lic_plate.append(ocr(char_img))
                        cv2.rectangle(plate_im,(x,y),(x+w,y+h),(0,0,255),1)
                    #cv2.imshow('bh',plate_im)

from tkinter import *
from tkinter import filedialog
import tkinter.font as font
root=Tk()
root.geometry('500x500')
root.resizable(width='false',height='false')
root.title('                                    License Plate Number Recognizer')
root.configure(bg='white')
lic_plate=[]

def open_file():
    num1=StringVar()
    root.filename=filedialog.askopenfilename()
    answer.config(textvariable=num1)
    label1.config(text='Image Loaded!')
def submit_file():     
    num=StringVar()
    img = cv2.imread(root.filename)
    threshold_img = preprocess(img)
    contours= extract_contours(threshold_img)
    cleanAndRead(img,contours)
    num.set("Predicted License Plate No.:\n\n ")
    for i in lic_plate:
        num.set(num.get() + " " + i)
    if(len(lic_plate)==0):
        num.set("No License Plate Found!")
    lic_plate.clear()
    #answer.delete('1.0',END)
    #answer.insert('1.0','hi!')
    answer.config(textvariable=num)
    
myFont = font.Font(family='Helvetica')
label=Label(root,text='License Plate Number Recognizer',width=30,height=3,bg='#00004d',fg='#00ffff',font='Helvetica 20 bold italic underline')
label.place(x=0,y=0)
#label.config(font=("Courier", 8))
upl=Button(root,text='Select Image',width=50,bg='#00ffff',fg='black',font='Helvetica 14',command=lambda:open_file())
upl.place(x=0,y=105)

label1=Label(root,text='Please Load an Image!',width=50,height=2,bg='#00004d',fg='#00ffff',font='Helvetica 14')
label1.place(x=0,y=147)

submit=Button(root,text='Extract Number',width=50,bg='#00ffff',fg='black',font='Helvetica 14',command=lambda:submit_file())
submit.place(x=0,y=200)
#label['font'] = myFont
answer=Label(root,height=12,width=50,font='Helvetica 14',bg='#00004d',fg='#00ffff')
answer.place(x=0,y=242)
root.mainloop()

cv2.destroyAllWindows()

Using TensorFlow backend.


Number of Contours found = 186
Number of Contours found in Clean Plate = 186
